# 🚀 Construction d'un Portefeuille Intelligent (Eightcap + GMV/CVaR)

Ce notebook implémente un pipeline complet de sélection et d'optimisation de portefeuille avec :
- Actions disponibles sur Eightcap (via MetaTrader5)
- Filtrage fondamental SMB et HML
- Estimation des rendements via Fama-French 3 facteurs
- Choix dynamique entre optimisation GMV ou CVaR
- (Exécution optionnelle sur MT5)

In [80]:
# 📦 Chargement des bibliothèques nécessaires
import MetaTrader5 as mt5
import yfinance as yf
import requests
import pandas as pd
import numpy as np
import time
import cvxpy as cp
from tqdm import tqdm
from statsmodels.api import OLS, add_constant

## 🎯 Étape 1 : Tickers Eightcap US uniquement

In [81]:
mt5.initialize()

data = [(s.path.split('\\')[0], s.path.split('\\')[1], s.path.split('\\')[2]) for s in mt5.symbols_get()]

df = pd.DataFrame(data, columns=['type','class','name'])

df.sort_values(['type','class'], ascending=[True,True], inplace=True)

tickers = list( dict.fromkeys(list(df[(df['type']=='Stock') & ((df['class']=='NYS') | (df['class']=='NAS'))]['name'].values)))

mt5.shutdown()

print(df[(df['type']=='Stock') & ((df['class']=='NYS') | (df['class']=='NAS'))]['name'].values)

print(f"{len(tickers)} tickers US récupérés.")

['AAPL' 'ADBE' 'AMAT' 'AMD' 'AMGN' 'AMZN' 'ASML' 'AVGO' 'BNTX' 'CHTR'
 'CMCSA' 'COST' 'CSCO' 'GOOGL' 'HON' 'HOOD' 'INTC' 'INTU' 'ISRG' 'MRNA'
 'MSFT' 'MU' 'NFLX' 'NVDA' 'PDD' 'PEP' 'PYPL' 'QCOM' 'SBUX' 'TMUS' 'TSLA'
 'TXN' 'ZM' 'SHOP' 'ROKU' 'ADSK' 'AEP' 'ALGN' 'ANSS' 'BIDU' 'BIIB' 'BKNG'
 'CDNS' 'CDW' 'CHKP' 'CPRT' 'CRWD' 'CSX' 'CTAS' 'CTSH' 'DLTR' 'DOCU' 'EA'
 'EBAY' 'EXC' 'FAST' 'FOX' 'GILD' 'IDXX' 'ILMN' 'INCY' 'KHC' 'KLAC' 'LRCX'
 'LULU' 'MAR' 'MCHP' 'MDLZ' 'MELI' 'MNST' 'MRVL' 'NTES' 'NXPI' 'ORLY'
 'PAYX' 'PCAR' 'REGN' 'ROST' 'SIRI' 'SNPS' 'SWKS' 'VRSK' 'VRSN' 'VRTX'
 'WBA' 'WDAY' 'XEL' 'ADI' 'JD_US' 'MTCH' 'TCOM' 'META' 'PLTR' 'GEHC' 'WBD'
 'ADP' 'APD' 'CME' 'EQIX' 'COIN' 'CEG' 'ABBV' 'ABT' 'ACN' 'BAC' 'BBY'
 'BHP_US' 'BMY' 'BRK.A' 'CAT' 'CRM' 'CVX' 'DAL' 'DHR' 'DIS' 'F' 'FCX' 'GE'
 'GM' 'GS' 'HD' 'JNJ' 'JPM' 'KO' 'LUV' 'MA' 'MCD' 'MDT' 'MRK' 'NEE' 'NKE'
 'NVO' 'NVS' 'ORCL' 'PFE' 'PG' 'PHM' 'PM' 'SAP_US' 'T' 'TM' 'TMO' 'TSM'
 'TSN' 'UNH' 'USB' 'V' 'VZ' 'WFC' 'WMT' 'XOM' 'AR' 'FD

## 📑 Étape 2 : Filtrage fondamental SMB / HML avec API FMP

### Parametrages API

In [82]:
API_KEY = "<YOUR_API_KEY>"


### Construire SMB et HML

In [83]:
fundamentals = {}

for t in tqdm(tickers, desc="Téléchargement des données fondamentales"):
    base = t.replace('.US', '')

    # Profil : Market Cap & Price
    url = f"https://financialmodelingprep.com/api/v3/profile/{base}?apikey={API_KEY}"
    try:
        r = requests.get(url)
        if r.ok and r.json():
            d = r.json()[0]
            fundamentals[t] = {
                "marketCap": d.get("mktCap"),
                "price": d.get("price")
            }
    except:
        continue
    time.sleep(1.2)

    # Ratios : Price-to-Book
    url = f"https://financialmodelingprep.com/api/v3/ratios-ttm/{base}?apikey={API_KEY}"
    try:
        r = requests.get(url)
        if r.ok and r.json():
            d = r.json()[0]
            pb = d.get("priceToBookRatioTTM")
            if pb and pb > 0:
                fundamentals[t]["bookValue"] = fundamentals[t]["price"] / pb
    except:
        continue
    time.sleep(1.2)    

df = pd.DataFrame(fundamentals).T.dropna()

df["B/M"] = 1 / df["bookValue"]
df["Size_rank"] = df["marketCap"].rank()
df["BM_rank"] = df["B/M"].rank(ascending=False)

print(df) 

small_cut = df["Size_rank"].quantile(0.3)
value_cut = df["BM_rank"].quantile(0.3)


filtered = df[(df["Size_rank"] <= small_cut) | (df["BM_rank"] <= value_cut)]
filtered_tickers = [t.replace('.US', '') for t in filtered.index]
print(f"{len(filtered_tickers)} tickers sélectionnés selon SMB / HML.")

Téléchargement des données fondamentales: 100%|██████████| 202/202 [09:28<00:00,  2.82s/it]

         marketCap     price   bookValue       B/M  Size_rank  BM_rank
AAPL  3.171020e+12  212.3100    4.456713  0.224381      114.0      6.0
ADBE  1.599401e+11  375.2700   26.504238  0.037730       82.0     59.0
AMAT  1.495375e+11  186.3400   23.468885  0.042610       78.0     52.0
AMD   2.234129e+11  137.7901   35.767976  0.027958       92.0     77.0
AMGN  1.586502e+11  295.0500   11.554405  0.086547       81.0     23.0
...            ...       ...         ...       ...        ...      ...
HD    3.685014e+11  370.3800    8.019153  0.124701      104.0     11.0
JNJ   3.742161e+11  155.5300   32.371492  0.030891      105.0     70.0
JPM   8.100353e+11  291.4750  124.840562  0.008010      108.0    113.0
KO    3.056893e+11   71.0200    6.084230  0.164359      100.0      7.0
LUV   1.918739e+10   33.6700   16.035959  0.062360       11.0     36.0

[116 rows x 6 columns]
62 tickers sélectionnés selon SMB / HML.


## 📈 Étape 3 : Téléchargement des prix et rendements

In [84]:
prices = yf.download(filtered_tickers, start="2024-01-01", end="2025-05-23")["Close"]
returns = prices.pct_change().dropna()
returns = returns.dropna(axis=1)
returns

[*********************100%***********************]  62 of 62 completed


Ticker,AAPL,ABBV,ADP,ADSK,ALGN,AMGN,ANSS,AVGO,BBY,BIDU,...,ROST,SHOP,SIRI,SWKS,TCOM,VRSK,WBA,WBD,XEL,ZM
Date,,,,,,,,,,,,,,,,,,,,,
2024-01-03,-0.007488,0.004004,-0.003899,-0.029600,-0.047226,0.011097,-0.027705,-0.024692,-0.023686,0.020375,...,-0.021136,-0.027225,-0.009107,-0.035744,0.019910,-0.005379,-0.040525,-0.030875,0.001733,-0.028633
2024-01-04,-0.012700,0.006232,0.004946,0.007615,0.020061,0.008248,0.002739,-0.009040,-0.001723,0.005353,...,0.003710,0.022278,-0.001838,-0.018915,0.057190,0.005281,-0.051232,0.004425,0.002359,-0.004020
2024-01-05,-0.004013,0.004212,0.006633,0.002621,0.012703,-0.000561,-0.000232,0.000257,0.007038,-0.001437,...,-0.005692,0.014846,0.005525,0.000484,-0.005202,-0.017861,0.030915,-0.013216,0.000000,0.000897
2024-01-08,0.024175,-0.004379,0.006590,0.025836,0.034986,0.026007,0.013224,0.024369,-0.003824,-0.002116,...,0.009740,0.042679,0.003663,0.027113,-0.010196,-0.004236,0.024790,0.017857,-0.002354,0.021356
2024-01-09,-0.002263,0.005451,-0.006504,0.002166,0.013069,-0.011644,0.015260,0.007108,-0.005030,-0.014419,...,0.000663,0.031149,-0.014599,-0.014047,-0.011622,0.008637,-0.010534,-0.032456,-0.004876,-0.001170
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-05-16,-0.000899,0.012880,0.011144,0.007776,0.008408,0.011511,0.009053,-0.017323,0.013180,0.001682,...,0.006781,0.004900,0.016547,0.014389,-0.008243,0.007190,0.009839,0.002188,0.015717,0.009183
2025-05-19,-0.011739,0.009184,0.011241,-0.007313,-0.018323,0.012628,-0.007645,0.008836,-0.029810,-0.001007,...,0.008763,-0.014898,-0.011439,-0.005456,0.032784,0.015441,-0.003543,-0.012009,-0.002349,-0.015481
2025-05-20,-0.009196,-0.004631,-0.000495,-0.000203,-0.012227,-0.003045,0.005872,0.004553,-0.006285,0.001008,...,-0.000130,-0.024473,0.004450,0.004937,-0.055440,-0.003977,-0.001778,0.019889,0.001523,-0.002521


## 📊 Étape 4 : Estimation Fama-French 3

In [85]:
factors = pd.read_csv("F-F_Research_Data_Factors_daily.csv")
factors = factors.rename(columns={'Mkt-RF': 'MKT'})
factors['Date'] = pd.to_datetime(factors['Date'], format='%Y%m%d')
factors = factors.set_index('Date') / 100
factors = factors[['MKT', 'SMB', 'HML', 'RF']]

print(factors)

returns, factors = returns.align(factors, join='inner', axis=0)

mu = []
for asset in returns.columns:
    y = returns[asset] - factors['RF']
    X = add_constant(factors[['MKT', 'SMB', 'HML']])
    model = OLS(y, X).fit()
    mu.append(model.predict(X.mean()).iloc[0] + factors['RF'].mean())


mu = np.array(mu)
cov = returns.cov()

               MKT     SMB     HML       RF
Date                                       
1926-07-01  0.0009 -0.0025 -0.0027  0.00009
1926-07-02  0.0045 -0.0033 -0.0006  0.00009
1926-07-06  0.0017  0.0030 -0.0039  0.00009
1926-07-07  0.0009 -0.0058  0.0002  0.00009
1926-07-08  0.0022 -0.0038  0.0019  0.00009
...            ...     ...     ...      ...
2025-03-25  0.0012 -0.0094  0.0005  0.00016
2025-03-26 -0.0122 -0.0048  0.0152  0.00016
2025-03-27 -0.0042  0.0019  0.0030  0.00016
2025-03-28 -0.0207 -0.0048  0.0040  0.00016
2025-03-31  0.0040 -0.0108  0.0037  0.00016

[25961 rows x 4 columns]


## ⚖️ Étape 5 : Optimisation dynamique GMV ou CVaR

In [86]:
vol_threshold = 0.02
market_vol = factors['MKT'].rolling(20).std().iloc[-1]

if market_vol > vol_threshold:
    print("🔺 Volatilité élevée – CVaR")
    R = returns.values
    w = cp.Variable(R.shape[1])
    z = cp.Variable(R.shape[0])
    VaR = cp.Variable()
    alpha = 0.05
    obj = cp.Minimize(VaR + (1 / (alpha * R.shape[0])) * cp.sum(z))
    constraints = [cp.sum(w) == 1, w >= 0, z >= 0, z >= -R @ w - VaR]
    cp.Problem(obj, constraints).solve()
    weights = w.value
else:
    print("🟢 Volatilité normale – GMV")
    w = cp.Variable(len(mu))
    obj = cp.Minimize(cp.quad_form(w, cov.values))
    constraints = [cp.sum(w) == 1, w >= 0]
    cp.Problem(obj, constraints).solve()
    weights = w.value

🟢 Volatilité normale – GMV


## 🧾 Étape 6 : Envoi automatique des ordres sur Eightcap (via MetaTrader 5)

In [88]:
# 🚀 Étape 6 (optionnelle) : Envoi des ordres sur MetaTrader 5 (Eightcap)
# Assurez-vous que MetaTrader 5 est lancé et connecté au bon compte

capital = 10000  # Capital à investir
mt5.initialize()

for symbol, weight in zip(returns.columns, weights):
    eightcap_symbol = symbol
    info = mt5.symbol_info_tick(eightcap_symbol)
    if info is None:
        print(f"⛔️ {eightcap_symbol} non disponible avec weight={weight}")
        continue
    lot = round((weight * capital / 100) / info.ask, 2)
    request = {
        "action": mt5.TRADE_ACTION_DEAL,
        "symbol": eightcap_symbol,
        "volume": lot,
        "type": mt5.ORDER_TYPE_BUY,
        "price": info.ask,
        "deviation": 20,
        "magic": 20250523,
        "comment": "Portefeuille Auto CVaR/GMV",
        "type_time": mt5.ORDER_TIME_GTC,
        "type_filling": mt5.ORDER_FILLING_IOC,
    }
    result = mt5.order_send(request)
    print(f"✅ {eightcap_symbol}: volume {lot} envoyé ({result.retcode})")

mt5.shutdown()

✅ AAPL: volume 0.01 envoyé (10009)
✅ ABBV: volume 0.02 envoyé (10009)
✅ ADP: volume 0.02 envoyé (10009)
✅ ADSK: volume -0.0 envoyé (10014)
✅ ALGN: volume -0.0 envoyé (10014)
✅ AMGN: volume 0.0 envoyé (10014)
✅ ANSS: volume 0.02 envoyé (10009)
✅ AVGO: volume 0.0 envoyé (10014)
✅ BBY: volume -0.0 envoyé (10014)
✅ BIDU: volume 0.0 envoyé (10014)
✅ BIIB: volume 0.0 envoyé (10014)
✅ BMY: volume 0.03 envoyé (10009)
✅ BNTX: volume -0.0 envoyé (10014)
✅ CDW: volume 0.01 envoyé (10009)
✅ CHKP: volume 0.01 envoyé (10009)
✅ CPRT: volume -0.0 envoyé (10014)
✅ CRWD: volume 0.0 envoyé (10014)
✅ CSCO: volume 0.03 envoyé (10009)
✅ CSX: volume 0.0 envoyé (10014)
✅ CTAS: volume -0.0 envoyé (10014)
✅ CTSH: volume -0.0 envoyé (10014)
✅ DAL: volume -0.0 envoyé (10014)
✅ DLTR: volume 0.01 envoyé (10009)
✅ DOCU: volume 0.01 envoyé (10009)
✅ EA: volume 0.02 envoyé (10009)
✅ EBAY: volume -0.0 envoyé (10014)
✅ EXC: volume 0.27 envoyé (10009)
✅ F: volume 0.0 envoyé (10014)
✅ FAST: volume 0.0 envoyé (10014)
✅ FCX

True